# Using SpecpolFlow: Example TKTKTK
The below tutorial has been developed for the MOBSTER Collaboration. The goal of this tutorial is to take a user from raw .s files to LSD profiles and Bz calculations. The structure of this notebook is as follows:

0. Importing nessecary packages
1. Explanination on converting telescope outputs (`.fits`) to spectrum files (`.s`). (This is not done in this tutorial, the `.s` files are provided)
2. Normalizing the spectrum with `normPlot`
3. Gathering stellar parameters and explaination of VALD line lists. (The parameters and line list is provided)
4. Creating the line mask from the VALD line list and removing contaminated or unwanted lines.
5. Creating the LSD profile
6. Calculating the longitudinal magnetic field (Bz)
7. Looping over multiple observations
8. LSD Sonification


## 0. Importing specpolFlow and other packages
The below block of code may output text such as "loading specpolFlow package" or "loading LSDpy package"; this an indication the packages are loading. If these messages do not appear, the packages may already be imported.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import specpolFlow as pol
import LSDpy

## 1. Converting Files
Before starting the tutorial, it is important to convert the files to a useful formats. We have provided .s files for TKTKTK but when you are working from your own files, please convert to .s files using the tutorial: `1-ConvertToSFiles_Tutorial.ipynb`.

## 2. Normalizing the Spectrum with `normPlot`
We need to normalize the spectrum before applying a line mask and creating an LSD profile. To normalize the spectrum, we will use [normPlot](https://github.com/folsomcp/normPlot); this program is designed for continuum normalizing on reduced 1D stellar spectra.

**TODO**

## 3. Gathering Stellar Parameters and Obtaining the VALD line list

1. Collect .s file. For this example, we provide the files (TKTKTK). 
2. Collect important stellar parameters ($T_{\text{eff}}$, log $_g$ , $v\sin i$, observation specific radial velocities). For this star, these are provided below: 
    * $T_{\text{eff}} = TKTKTK$ K
    * $\log g = TKTKTK$ (cgs)
    * $v\sin i  = TKTKTK$ km $\text{s}^{-1}$
    * $\text{RV} = TKTKTK$ km $\text{s}^{-1}$
3. Next we will need to obtain a list of atomic and molecular transition parameters for TKTKTK from the Vienna Atomic Line Database (VALD; [Ryabchikova et al. 2015](https://ui.adsabs.harvard.edu/abs/2015PhyS...90e4005R/abstract)). **The long list for this star is provided (TKTKTK)**, however, to retrieve a VALD long list, one can click the "Extract Stellar" button on the [VALD website](http://vald.astro.uu.se/).  Note that you need to register an email to access the site. The following input was used to obtain the example long list:
    * Starting wavelength: TKTKTK  $\text{\AA}$
    * Ending wavelength: TKTKTK  $\text{\AA}$
    * Detection threshold: 0.01 (line depths shallower than this threshold are not included)
    * Microturbulence: TKTKTK km $\text{s}^{-1}$
    * $T_{\text{eff}} = TKTKTK$ K
    * $\log g = TKTKTK$ (cgs)
    * Chemical Composition: TKTKTK [Fe/H] (Solar)
    * Long format
    * Linelist configuration: Default

The long list file is a text file consisting of a header line containing information on the wavelength range of the list, as well as other information about the content of the file. Each spectral line has four rows of information in the file: the first row contains the most relevant information for making the mask, but the format is rather unpleasant to use. The line mask has a much more simplistic format that makes it easier to parse and manipulate. Additionally, the line mask calculates the effective Landé factor for each of the lines.

In [ ]:
spectrum = 'MOBSTER_tutorialfiles/TKTKTK_test_1.s' # location of .s file
long_list = 'MOBSTER_tutorialfiles/LongList_TKTKTK.dat' # location of .dat file

## 4. Create and Clean a Mask

The mask tells us the location and depth of all lines in a spectrum but does not tell us about the shape of the lines of the spectrum as a whole. We need a mask to help us weigh each spectral line in the spectrum so that they can be averaged together in an LSD. Given a spectrum and a mask, we can deconvolve the spectrum with the mask to generate an LSD profile.

To create a mask we will use the `make_mask` function. The function has the following inputs:
* lineListFile - name of the file containing the line list
* maskFile - name of the file to write the output mask to
* depthCutoff - only include lines in the mask that are deeper than this value
* wlStart - optional, use only lines with wavelengths above this
* wlEnd - optional, use only lines with wavelengths below this
* landeStart - optional, use only lines with effective Lande factors above this
* landeEnd - optional, use only lines with effective Lande factors below this
* elementsUsed - optional, list of elements to include in the mask creation
* elementsExclude - optional, list of elements to exlude from the mask creation
* atomsOnly - only include atomic lines (no molecular lines and no H-lines)
* includeNoLande - include lines in the mask even if they don't have an effective Lande factor given
* DefaultLande - the value for the effective Lande factor to use in the case that a line has no listed effective Lande factor 

For most cases you will only use the top three inputs and `atomsOnly`. Note that the hydrogen lines are automatically exluded when `atomsOnly` is `True`. This is done since the hydrogen lines have different shape than all the other lines in the spectrum due to their broad wings. The input line list is a VALD line list file and is obtained from the VALD website. More details about VALD is given in `OneObservationFlow_Tutorial.ipynb`. The `make_mask` function will automatically attempt to calculate the effective Lande factor for lines where it is not given, however if one is unable to be calculated it will be excluded if `includeNoLande` is `False` or it will equal `DefaultLande` if `includeNoLande` is `True`.

In the example case used below, we are using all atomic lines in the line list except ones without effective Lande factors. We also remove all lines with a depth smaller than the `depthCutoff` of 0.02. The Hydrogen lines are also removed by default, however these lines have very broad wings that lead to blending with nearby lines. As a result, we will need to remove nearby blended lines in the next subsection.


In [ ]:
file_output = 'MOBSTER_tutorialfiles/test_output/TKTKTK_depth_0.02.mask'
mask = pol.make_mask(long_list, file_output, depthCutoff = 0.02, atomsOnly = True)

### 4.1 Removing Balmer and Telluric Lines Manually

There are certain spectral regions that the user may want to exclude from the mask. Commonly these excluded regions include regions of telluric or atmospheric contamination as well as the Balmer series. The Balmer lines have large pressure broadened wings, the lines overlapping with these wings need to be removed. For this example we remove lines within $\pm$ 400 km/s of the Balmer lines. The user can also specify additional regions of their choice to remove. Removal of specific elements is better handled through the `mask` object as described in section 4.3.

First we will create a `.dat` file with all of the regions we want to exclude from the mask. All spectral lines within these regions will have their `.iuse` set to 0 indicating that the line is not to be used. 



In [6]:
excluded_lines = {'start':[655.405353,485.491365,433.470866,409.622728,396.480287,360.0,587.5,627.5,686.0,717.0,759.0,790.0,809.0], 
     'stop': [657.156647,486.788635,434.629134,410.717272,397.539713,392.0,592.0,632.5,705.3,735.0,771.0,795.0,990.0], 
     'type': ['Halpha','Hbeta','Hgamma','Hdelta','Hepsilon','Hjump','telluric','telluric','telluric','telluric','telluric','telluric','telluric']}
excluded_lines = pd.DataFrame(data = excluded_lines)

In [ ]:
excluded_lines.to_csv('ExcludeMaskRegionsClass_tutorialfiles/MOBSTER_custom.dat', header = False, index = False, sep = '\t')
excluded_lines = pol.read_exclude_mask_regions('ExcludeMaskRegionsClass_tutorialfiles/MOBSTER_custom.dat')

clean_Mask_filename = 'MOBSTER_tutorialfiles/test_output/TKTKTK_manual_clean.mask' # location of the output file
mask_clean = mask.clean(excluded_lines).save(clean_Mask_filename)

### 4.2 Using the Default Function

`SpecpolFlow` includes default excluded regions for ease of use. The two functions given below define the default telluric regions as well as calculating regions around the Balmer lines to be removed given an input `velrange`. 

For this example we will use a velrange of 400 km/s.

In [ ]:
velrange = TKTKTK # units are in km/s
excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()
pd.DataFrame(excluded_regions.to_dict()) # visualizing dataframe w/ excluded regions

In [ ]:
clean_Mask_filename = 'MOBSTER_tutorialfiles/test_output/TKTKTK_depth_0.02_clean.mask' # location of the output file

mask.clean(excluded_regions).save(clean_Mask_filename) # run cleanMask fuction and save

### 4.3 Stucture of the mask object and removing the specific lines

The mask object has a few key attributes:
* wl - list of wavelengths (in nm) corresponding to every line in the mask
* element - list of elements and ionization states for each line. The structure of this output is a float. The number on the left of the decimal is the atomic number of the element. The right number is the ionization state of the element. For example neutral oxygen (OI) is given by `8.` and singly ionized oxygen (OII) is given by `8.01`
* depth - list of depths to each line
* lande - list of Lande factors for each line
* iuse - list of values (0 or 1) for each line. A 0 means the line is not being used.

We can also remove specific spectral lines by setting `.iuse` equal to 0 for those elements/ions. Below we remove all Helium lines regardless of ionization state. `mask.element` outputs a list with all elements/ions in the mask. 

In [ ]:
mask_noHe=mask
mask_noHe[np.where((mask_noHe.element>=2.0) & (mask_noHe.element<3.))].iuse=0

In [ ]:
mask_noHe.element.min()

## 5. Create LSD Profile

Least-Squares Deconvolution (LSD) is a cross-correlation technique for computing the weighted average of selected spectral lines ([Donati et al. 1992](https://ui.adsabs.harvard.edu/abs/1997MNRAS.291..658D/abstract)). We use the default conditions:
* normDepth = 0.2 — normalized line depth
* normLande = 1.2 — normalized effective Lande factor
* normWave = 500.0 — normalized wavelength

To capture the entire line profile, the range of the LSD profile (`velStart` and `velEnd`) is set to be $\pm$ 100 km $s^{-1}$ for this star. Additionally, the pixel size should be set relative to the resolution of the original data; for ESPaDOnS data, one should not go below $2.6$ km/s per pixel. In a case where the line profile is very broad, it may be advantageous to use larger pixels but make sure the profile has enough data. For our example, we use the ESPaDOnS lower threshold of $2.6$ km $s^{-1}$ per pixel.

To calculate the LSD profile, we call the `run_lsdpy` function, specify the file (.s file) and mask (.mask file). In the below code, we also input a name and location of the outfile (.lsd). For all outputs from this function and additional information on the utilities of the LSD class, see [LSDpy](https://github.com/folsomcp/LSDpy).

In [ ]:
outfile = 'MOBSTER_tutorialfiles/test_output/TKTKTK_test_1.lsd'

lsd, mod = pol.run_lsdpy(obs = spectrum, mask = clean_Mask_filename, outName = outfile, 
           velStart = -100.0, velEnd = 100.0, velPixel = 2.6, 
           normDepth = 0.2, normLande = 1.2, normWave = 500.0)

## 6. Calculate Bz

To calculate Bz we will be using the `lsd.calc_bz` function which takes in the following:
* cog - the center of gravity; this can either be set manually or calculated from one of the built in functions below
    * I (from Stokes I)
    * IV (from Stokes I times V)
    * V (from Stokes V)
    * min (minimum of profile)
* velrange - the total range over which will be considered in the Bz calculation; this includes the line itself as well as a little extra on either side
* plot - a flag that determines if an output plot will be automatically generated
* bzwidth - the range over which the Bz will be calculated; this should just include the line itself (if given a single number it will center the range on the cog value)

In the first example (6.1) we are manually setting the velocity range and Bz width, however it is often useful to set the velocity range slightly larger than the $v\sin i$ and shifted to be centered on the cog value. By these standards, for this example, velrange _*would*_ be `velrange=[12-1.5*vsini,12+1.5*vsini]` and bzwidth _*would*_ be `bzwidth=vsini`. 

In the second example (6.2) the cog value is being set automatically using the I method. 

The function will then output a dictionary with the following outputs:
* Ic: continuum value used for normalization
* Cog: the center of gravity value
* Bzwidth min (km $s^{-1}$): lower bound of the Bz width
* Bzwidth max (km $s^{-1}$): upper bound of Bz width
* V bz (G): Bz calculated from Stokes V profile
* V bz sig (G): standard deviation
* V FAP: false alarm probability [FAP (Donati et al. 1997)](https://ui.adsabs.harvard.edu/abs/1997MNRAS.291..658D/abstract). Definite detection (DD) is defined as having a FAP $< 10^{-5}$. A non-detection (ND) is defined as having a FAP $> 10^{-3}$. FAPs between $10^{-5}$ and $10^{-3}$ are defined as a marginal detection (MD).
* Null calculations - N1 and N2 are two different methods for null profile calculations; N1 is most commonly used.
    * N1 bz (G): Bz calculated from the Null 1 profile
    * N1 bz sig (G): standard deviation
    * N1 FAP: false alarm probability of Null 1 Bz measurement
    * N2 bz (G): Bz calculated from the Null 2 profile
    * N2 bz sig (G): standard deviation
    * N2 FAP: false alarm probability of Null 2 Bz measurement
    
To see additional capabilities of the Bz function, see [SpecpolFlow](https://github.com/folsomcp/specpolFlow) or for a more detailed tutorial, see the [Bz tutorial](https://github.com/folsomcp/specpolFlow/blob/main/tutorials/CalculateBz.ipynb).

### 6.1 Using Manual Cog

In [ ]:
# Bz calculation using manual cog selection
vrad = TKTKTK
velrange = [TKTKTK,TKTKTK] # the velocity range over which the center line is found
bzwidth = TKTKTK # this is the width about the center line used to calculate Bz

lsd = pol.read_lsd(outfile)
Bz, fig = lsd.calc_bz(cog = vrad, velrange = velrange, plot = True, bzwidth = bzwidth)

### 6.2 Using Cog I

In [ ]:
# Bz calculation using manual cog selection
vrad = TKTKTK
velrange = [TKTKTK,TKTKTK] # the velocity range over which the center line is found
bzwidth = TKTKTK # this is the width about the center line used to calculate Bz

lsd = pol.read_lsd(outfile)
Bz, fig = lsd.calc_bz(cog = 'I', velrange = velrange, plot = True, bzwidth = bzwidth)

In [ ]:
pd.DataFrame(data=[Bz])

## 7. Looping over Multiple Observations

When a star has multiple observations, it is useful to make a loop over all of the spectra files rather than running each one individually. To obtain outputs of interest, only one cleaned line mask is necessary since all the spectra files are from the same star. With the cleaned mask and .s files, you must loop over the  `run_lsdpy` function to calculate LSD profiles for each inputed spectra file. We have provided additional files for TKTKTK: TKTKTK_test_2.s & TKTKTK_test_3.s.

In [ ]:
for i in range(TKTKTK):
    
    clean_Mask_filename = 'MOBSTER_tutorialfiles/test_output/TKTKTK_depth_0.02_clean.mask' #clean mask file
    obsfile = 'MOBSTER_tutorialfiles/TKTKTK_test_{}.s'.format(i+1) #input spectrum file
    lsdfile = 'MOBSTER_tutorialfiles/test_output/TKTKTK_test_{}.lsd'.format(i+1) #output file

    lsd, mod = pol.run_lsdpy(obs = obsfile, 
        mask = clean_Mask_filename, outName = lsdfile, 
        velStart = -100.0, velEnd = 100.0, velPixel = 2.6, 
        normDepth = 0.2, normLande = 1.2, normWave = 500.0,
        fLSDPlotImg=0)
    
    #plot outputs
    fig, ax = lsd.plot()
    fig.suptitle('TKTKTK' + '-'+ str(i+1),y = 0.92)
    plt.show()

## 8. Sonification of LSD Profile (If there is time)

In [1]:
import astronify as snd
from astropy.table import Table


WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter (if available).
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



In [2]:
def sono_lsd(lsd):
    ''' 
    Creates a sonification of a LSD profile object. 
    '''

    # The astronify package requires astropy tables. 
    # data_table_I = Table({"vel":lsd.vel, "Stokes":lsd.specI})
    # data_table_N1 = Table({"vel":lsd.vel, "Stokes":lsd.specN1})
    # data_table_V = Table({"vel":lsd.vel, "Stokes":lsd.specV})

    sigmaN = np.std(lsd.specN1) # The stdev of N1:
    maxV = np.max(np.abs(lsd.specV)) # The max of V
    # We clip the sound range at either 5 times the deviation of N
    # (in the case where StokesV is not detected) 
    # or to the max value of Stokes V
    clip = np.max([5*sigmaN, maxV])

    data_table = Table({"vel":lsd.vel, "Stokes":lsd.specI})
    data_soni = snd.series.SoniSeries(data_table, time_col='vel', val_col='Stokes')
    data_soni.note_spacing = 0.02
    data_soni.pitch_mapper.pitch_map_args["zero_point"] = 1.0
    data_soni.pitch_mapper.pitch_map_args['minmax_value'] = [0.5, 1.5]
    data_soni.pitch_mapper.pitch_map_args['pitch_range'] = [100, 700]
    data_soni.pitch_mapper.pitch_map_args['center_pitch'] = 400
    data_soni.sonify()
    data_soni.play() 
    data_soni.write('I.wav')

    for i, Stokes in enumerate([lsd.specN1, lsd.specV]):

        data_table = Table({"vel":lsd.vel, "Stokes":Stokes})
        data_soni = snd.series.SoniSeries(data_table, time_col='vel', val_col='Stokes')
        data_soni.note_spacing = 0.02
        data_soni.pitch_mapper.pitch_map_args["zero_point"] = 0.0
        data_soni.pitch_mapper.pitch_map_args['minmax_value'] = [-1*clip, clip]
        data_soni.pitch_mapper.pitch_map_args['pitch_range'] = [100, 700]
        data_soni.pitch_mapper.pitch_map_args['center_pitch'] = 400
        data_soni.sonify()
        data_soni.play() 
        if i==0:
            data_soni.write('N1.wav')
        else:
            data_soni.write('V.wav')

    
    return(data_soni)

In [ ]:
lsd = pol.read_lsd('CalculateBz_tutorialfiles/SampleLSD.s')
lsd = lsd[np.logical_and(lsd.vel>-50,lsd.vel<150)]
# lsd.specV = lsd.specN1
fig, ax = lsd.plot()


data_soni = sono_lsd(lsd)

data_soni.sonify()
data_soni.play() 
#data_soni.write('N1.wav')